# Library

In [ ]:
from library import *


2023-02-27 13:56:02.779307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 13:56:02.943728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 13:56:02.943767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-27 13:56:02.977598: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-27 13:56:04.832141: W tensorflow/stream_executor/pla

# Data importation

In [ ]:
data = pd.read_csv("Datasets/data.csv", sep=";")
data

,left_char,right_char,strength_left,strength_right,value_left,scenario,value_left_rep1,value_left_rep2,sex,age
0,2,6,7,3,8,4,-1,-1,0,57
1,6,5,3,6,1,2,-1,-1,0,57
2,6,3,3,8,1,3,-1,-1,0,57
3,7,6,10,3,0,9,-1,-1,0,57
4,3,5,8,6,5,7,-1,-1,0,57
...,...,...,...,...,...,...,...,...,...,...
8959,6,1,11,1,5,8,7,8,1,21
8960,5,2,14,11,7,5,6,7,1,21
8961,6,1,11,1,6,2,9,5,1,21
8962,1,4,1,11,5,7,3,2,1,21


# Set train and test dataset

In [ ]:
#set feature_names
feature_names = ["left_char","right_char","strength_left","strength_right","scenario","value_left_rep1","value_left_rep2","age","sex"]


In [ ]:
#get X and Y
X = data[feature_names]
Y = data.value_left

In [ ]:
#split in train & test dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

# Linear Regression

## Model train

In [ ]:
LR = LinearRegression()

params = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
}

grid_search_cv = GridSearchCV(LR, params, cv=5, verbose=1)
grid_search_cv


GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False],
                         'normalize': [True, False]},
             verbose=1)

In [ ]:
#model fitting
#grid_search_cv.fit(X_train, Y_train)
#best_LR = grid_search_cv.best_estimator_
#print("Best score : ", grid_search_cv.best_score_)
#print("Best: %f using %s" % (grid_search_cv.best_score_, grid_search_cv.best_params_))


Fitting 5 folds for each of 4 candidates, totalling 20 fits
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To rep

In [ ]:
#saving the model
filename = 'Model/LR.pkl'
#print("Sauvegarde du modèle dans ", filename)
#f = open(filename, "wb")
#pickle.dump(best_LR, f)
#f.close()

In [ ]:
# loading the model
LR = pickle.load(open(filename, 'rb'))
print(LR)

LinearRegression(normalize=True)


## Model test

In [ ]:
#Test on the test set
Y_test_predict = LR.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, Y_test_predict)))
r2 = r2_score(Y_test, Y_test_predict)


In [ ]:
#confusion matrix & report classification
print('performance of the model on the test basis')
print('--------------------------------------')
print('mean square error {}'.format(rmse))
print('R2 score {}'.format(r2))
Y_test_predict = Y_test_predict.astype(int)
print(
    classification_report(Y_test,
                          Y_test_predict,
                          output_dict=False,
                          target_names=[str(i) for i in range(max(Y_test)+1)]))

performance of the model on the test basis
--------------------------------------
mean square error 2.4378031231356045
R2 score 0.315275345276271
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       175
           1       0.26      0.10      0.14        94
           2       0.18      0.27      0.22       140
           3       0.16      0.33      0.22       154
           4       0.13      0.36      0.19       132
           5       0.40      0.34      0.37       427
           6       0.19      0.34      0.25       128
           7       0.29      0.29      0.29       164
           8       0.12      0.07      0.09       127
           9       0.50      0.08      0.14        61
          10       1.00      0.01      0.02       191

    accuracy                           0.22      1793
   macro avg       0.29      0.20      0.17      1793
weighted avg       0.32      0.22      0.20      1793



In [ ]:
#confusion matrix of prediction to within 1 class
Y_test = pd.concat([Y_test], ignore_index=True)
classification_report_opti(Y_test, Y_test_predict)

0 :  {'TN': 1590, 'FN': 164, 'FP': 28, 'TP': 11}
1 :  {'TN': 1489, 'FN': 52, 'FP': 210, 'TP': 42}
2 :  {'TN': 1175, 'FN': 53, 'FP': 478, 'TP': 87}
3 :  {'TN': 863, 'FN': 26, 'FP': 776, 'TP': 128}
4 :  {'TN': 716, 'FN': 22, 'FP': 945, 'TP': 110}
5 :  {'TN': 705, 'FN': 119, 'FP': 661, 'TP': 308}
6 :  {'TN': 978, 'FN': 52, 'FP': 687, 'TP': 76}
7 :  {'TN': 1249, 'FN': 70, 'FP': 380, 'TP': 94}
8 :  {'TN': 1465, 'FN': 75, 'FP': 201, 'TP': 52}
9 :  {'TN': 1664, 'FN': 42, 'FP': 68, 'TP': 19}
10 :  {'TN': 1594, 'FN': 187, 'FP': 8, 'TP': 4}
class | precision | recall | f1-score | accuracy
0     |  0.282     |  0.063  |  0.103    |  0.893
1     |  0.167     |  0.447  |  0.243    |  0.854
2     |  0.154     |  0.621  |  0.247    |  0.704
3     |  0.142     |  0.831  |  0.243    |  0.553
4     |  0.104     |  0.833  |  0.185    |  0.461
5     |  0.318     |  0.721  |  0.441    |  0.565
6     |  0.1     |  0.594  |  0.171    |  0.588
7     |  0.198     |  0.573  |  0.294    |  0.749
8     |  0.206  

[0.893, 0.854, 0.704, 0.553, 0.461, 0.565, 0.588, 0.749, 0.846, 0.939, 0.891]

In [ ]:
#features importances
coef = LR.coef_
coef

array([-3.79374297e-02,  3.64104381e-02,  2.27665757e-01, -2.30951387e-01,
        4.53044274e-05,  2.79384148e-01,  1.69535808e-02,  4.79693134e-04,
        1.33545806e-02])

# Average accuracy on 10 train/test dataset different

In [ ]:
display(LR.get_params())
column = ['0 acc_0','0 acc_1','1 acc_0','1 acc_1','2 acc_0','2 acc_1','3 acc_0','3 acc_1','4 acc_0','4 acc_1','5 acc_0','5 acc_1','6 acc_0','6 acc_1','7 acc_0','7 acc_1','8 acc_0','8 acc_1','9 acc_0','9 acc_1','10 acc_0','10 acc_1']
class_accuracy = pd.DataFrame(columns=column)
for i in tqdm(range(10)):
    print(" Modèle n°"+str(i+1)+" : STARTED \n")
    rand_int = random.randint(2, 1000)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=rand_int)

    #model train
    LR_test = LinearRegression()
    LR_test.fit(X_train, Y_train)

    #model test
    Y_test_predict = LR_test.predict(X_test)
    rmse = (np.sqrt(mean_squared_error(Y_test, Y_test_predict)))
    r2 = r2_score(Y_test, Y_test_predict)

    #classification
    vector = np.vectorize(np.int_)
    acc_0, prec_0 = accuracy_inclasses(Y_test, vector(Y_test_predict))
    print('performance of the model on the test basis')
    print('--------------------------------------')
    print('mean square error {}'.format(rmse))
    print('R2 score {}'.format(r2))
    Y_test_predict = Y_test_predict.astype(int)
    print(
    classification_report(Y_test,
                          Y_test_predict,
                          output_dict=False,
                          target_names=[str(i) for i in range(max(Y_test) + 1)]))
    Y_test = pd.concat([Y_test], ignore_index=True)
    acc_1 = classification_report_opti(Y_test, Y_test_predict)
    
    for j in range(11):
        class_accuracy.at[i, str(j) + " acc_0"] = acc_0[j]
        class_accuracy.at[i, str(j) + " acc_1"] = acc_1[j]
    print("\n")
    print(" Modèle n°"+str(i+1)+" : FINISHED \n")



{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': True,
 'positive': False}

0 :  {'TN': 1625, 'FN': 141, 'FP': 24, 'TP': 3}
1 :  {'TN': 1537, 'FN': 48, 'FP': 165, 'TP': 43}
2 :  {'TN': 1205, 'FN': 45, 'FP': 455, 'TP': 88}
3 :  {'TN': 874, 'FN': 25, 'FP': 751, 'TP': 143}
4 :  {'TN': 696, 'FN': 19, 'FP': 965, 'TP': 113}
5 :  {'TN': 690, 'FN': 116, 'FP': 667, 'TP': 320}
6 :  {'TN': 980, 'FN': 49, 'FP': 683, 'TP': 81}
7 :  {'TN': 1236, 'FN': 70, 'FP': 388, 'TP': 99}
8 :  {'TN': 1446, 'FN': 87, 'FP': 201, 'TP': 59}
9 :  {'TN': 1629, 'FN': 56, 'FP': 82, 'TP': 26}
10 :  {'TN': 1621, 'FN': 152, 'FP': 10, 'TP': 10}
class | precision | recall | f1-score | accuracy
0     |  0.111     |  0.021  |  0.035    |  0.908
1     |  0.207     |  0.473  |  0.288    |  0.881
2     |  0.162     |  0.662  |  0.26    |  0.721
3     |  0.16     |  0.851  |  0.269    |  0.567
4     |  0.105     |  0.856  |  0.187    |  0.451
5     |  0.324     |  0.734  |  0.45    |  0.563
6     |  0.106     |  0.623  |  0.181    |  0.592
7     |  0.203     |  0.586  |  0.302    |  0.745
8     |  0.227  

In [ ]:
round(class_accuracy.mean() * 100, 1)

0 acc_0     90.8
0 acc_1     90.1
1 acc_0     94.6
1 acc_1     87.7
2 acc_0     85.6
2 acc_1     71.5
3 acc_0     77.9
3 acc_1     55.3
4 acc_0     76.1
4 acc_1     43.7
5 acc_0     73.3
5 acc_1     55.6
6 acc_0     84.7
6 acc_1     59.2
7 acc_0     86.8
7 acc_1     75.1
8 acc_0     89.5
8 acc_1     84.7
9 acc_0     95.1
9 acc_1     92.7
10 acc_0    91.2
10 acc_1    91.1
dtype: float64

In [ ]:
models_df = pd.DataFrame({'basic':[ 22, 22, 22, 22, 23, 22, 20, 23, 24, 20],
        'opti': [ 73.4, 73.4, 73.4, 73.5, 73.5, 73.3, 73.2, 73.4, 73.6, 73.3]} )
print("basc: ",models_df.basic.mean(),"\nopti: ",models_df.opti.mean())

basc:  22.0 
opti:  73.4


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df420847-9b7d-4085-bb2b-2d13083511fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>